<a href="https://colab.research.google.com/github/marianawebb/basic-repo/blob/main/WLDAS_ProcessNetCDFs_05_09_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
%%capture
# Update spatial packages from colab default
!mamba install -q -c conda-forge cartopy
!mamba install -q -c conda-forge geopandas
!mamba install -q -c conda-forge rioxarray
# !mamba install -q -c conda-forge nc-time-axis
!mamba install -q -c conda-forge regionmask
!mamba install -q -c conda-forge dask 

# !pip install rioxarray
# !pip install geopandas
# !pip install regionmask

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import matplotlib.pyplot as plt
import regionmask
import xarray as xr
import os
import glob
import numpy as np
import pandas as pd
import rioxarray as rio
import csv
import dask
from dask.diagnostics import ProgressBar
from numba import jit, njit, vectorize, cuda
 
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/Dissertation/')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/Dissertation'

In [ ]:
def preprocessNetCDFs(raw_nc):
  print(raw_nc.time.values[0])
  # add lat and long as coordinates for WLDAS xarrays
  file = open("Data/WLDAS_LatLon/WLDAS_Lats.csv", "r")
  csv_reader = csv.reader(file)
  lats = []
  for row in csv_reader:
       lats.append(row[0])
  lats = lats[1:]
  lats = [float(i) for i in lats]

  file = open("Data/WLDAS_LatLon/WLDAS_Lons.csv", "r")
  csv_reader = csv.reader(file)
  lons = []
  for row in csv_reader:
       lons.append(row[0])
  lons = lons[1:]
  lons = [float(i) for i in lons]

  # drop_nc = raw_nc.drop(['Swnet_tavg','Lwnet_tavg','Qle_tavg','Qh_tavg','Qg_tavg','VegT_tavg','BareSoilT_tavg','RadT_tavg','ECanop_tavg','TVeg_tavg','ESoil_tavg','WaterTableD_tavg','Wind_f_tavg','Rainf_f_tavg','Psurf_f_tavg','SWdown_f_tavg','LWdown_f_tavg'])
  drop_nc = raw_nc.drop(['Swnet_tavg', 'Lwnet_tavg', 'Qle_tavg', 'Qh_tavg', 'Qg_tavg', 'VegT_tavg', 'BareSoilT_tavg', 'AvgSurfT_tavg', 'RadT_tavg', 'SnowDepth_tavg', 'SoilTemp_tavg', 'TVeg_tavg', 'ESoil_tavg', 'SubSnow_tavg', 'WaterTableD_tavg', 'SnowCover_tavg', 'Rainf_f_tavg', 'Psurf_f_tavg', 'SWdown_f_tavg', 'LWdown_f_tavg'])


  coord_nc = drop_nc.assign(north_south=lats, east_west=lons).drop(['lat', 'lon'])
  coord_nc = coord_nc.rename({'north_south':'lat', 'east_west':'lon'}).rio.write_crs("epsg:4326", inplace=True) ##assuming WGS84???


  #crop to lat lon bounds of watersheds
  crop_nc = coord_nc.sel(
       lon = slice(-124.5, -117.5),
       lat = slice(34.0, 49.0)
  )


  # make each soil moisture depth into a separate data variable
  soil_moist = crop_nc.SoilMoist_tavg
  soil_moist_list = []
  for i in np.arange(0,4):
    temp = soil_moist.isel({'SoilMoist_profiles':i})
    temp = temp.rename(temp.name + "_" + str(i+1))
    soil_moist_list.append(temp)

  soil_moist_list[0]
  soil_moist_merge = xr.merge(soil_moist_list)

  drop_nc = crop_nc.drop(['SoilMoist_tavg'])
  nc = xr.merge([drop_nc, soil_moist_merge])

  return nc

In [ ]:
# set year of interest
# yr = 1980
yr_list = np.arange(2000,2016)

# for loop for 5 years of data
for yr in yr_list:

  # load in shapefiles for each watershed
  watersheds_shp = gpd.read_file("Data/Shapefiles/watershed_boundaries.shp") 
  watersheds_shp['GAGE_ID'] = watersheds_shp['GAGE_ID'].astype('int64')


  #create a list containing a gdp object for each watershed
  watersheds_shp_list = []
  for i in watersheds_shp.iterrows():
    watersheds_shp_list.append(i)


  # load in list of one year of WLDAS netCDF
  months_list = [str(x).zfill(2) for x in np.arange(1,13)]

  for month in months_list:
    print(month)
    WLDAS_filelist = glob.glob('Data/WLDAS_Full/'+str(yr)+'/LIS_HIST_'+str(yr)+month+'*.nc')

    # checking if year already processed
    ym = str(yr)+month
    processed_files = glob.glob("Data/WLDAS_Processed/*.csv") + glob.glob("Data/WLDAS_ProcessedUNR/*.csv")
    processed_ym = [x[-10:-4] for x in processed_files]
    if ym in processed_ym:
      print(ym+" already processed")
      continue

    # open netCDFs at xarray objects
    nc = xr.open_mfdataset(WLDAS_filelist, parallel=False) #this method creates a dask array which I don't know how to work with :(
    nc = preprocessNetCDFs(nc)

    # create watersheds mask
    watersheds_mask = regionmask.mask_3D_geopandas(watersheds_shp, nc, numbers = "GAGE_ID", overlap = True)


    # mask WLDAS data to watershed extents
    print("masking...")
    masked_WLDAS = nc.where(watersheds_mask)


    #calculate cummulative and mean values for each watershed for each day
    print("calculating watershed means...")
    WLDAS_means = masked_WLDAS.groupby("region").mean(['lat', 'lon'])
    means_df = WLDAS_means.to_dask_dataframe()
    means_df.columns = means_df.columns.str.replace('_tavg', '')
    means_df = means_df.rename(columns=dict(zip(means_df.columns[3:], means_df.columns[3:] + '_mean')))
    # means_df = means_df.iloc[:,:-1]

    print("calculating watershed sums...")
    WLDAS_sums = masked_WLDAS.groupby("region").sum(['lat', 'lon'])
    sums_df = WLDAS_sums.to_dask_dataframe()
    sums_df.columns = sums_df.columns.str.replace('_tavg', '')
    sums_df = sums_df.rename(columns=dict(zip(sums_df.columns[3:], sums_df.columns[3:] + '_sum')))
    # sums_df = sums_df.iloc[:,:-1]


    # merge sum and mean dataframes
    print("merging into full dataframe")
    full_df = means_df.merge(
        sums_df, 
        how="left", 
        on=['region', 'time', 'spatial_ref']
    )

    print("computing dask...")
    with ProgressBar():
      compute_df = full_df.compute()

    print("exporting...")
    export_location = 'Data/WLDAS_ProcessedUNR/' + 'WLDAS_' +str(yr)+month+ '.csv'
    compute_df.to_csv(export_location, index = True)
    print('exported data for '+ str(yr))

01
200001 already processed
02
200002 already processed
03
200003 already processed
04
200004 already processed
05
200005 already processed
06
200006 already processed
07
200007 already processed
08
200008 already processed
09
200009 already processed
10
200010 already processed
11
200011 already processed
12
200012 already processed
01
200101 already processed
02
200102 already processed
03
200103 already processed
04
200104 already processed
05
200105 already processed
06
200106 already processed
07
200107 already processed
08
200108 already processed
09
200109 already processed
10
200110 already processed
11
200111 already processed
12
200112 already processed
01
200201 already processed
02
200202 already processed
03
200203 already processed
04
200204 already processed
05
200205 already processed
06
200206 already processed
07
200207 already processed
08
200208 already processed
09
200209 already processed
10
200210 already processed
11
200211 already processed
12
200212 already pr

Using these tutorials to help with making the multi-polygon mask
http://www.matteodefelice.name/post/aggregating-gridded-data/
https://www.earthdatascience.org/courses/use-data-open-source-python/hierarchical-data-formats-hdf/subset-netcdf4-climate-data-spatially-aoi/
https://www.guillaumedueymes.com/post/shapefiles_country/

For info on how the vector watershed outlines are rasterized: https://regionmask.readthedocs.io/en/stable/notebooks/method.html

Basically, if the 1 km centroid falls in the watershed, we include it in the mask

In [ ]:
sys.getsizeof(compute_df)

906224